# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [36]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [64]:
# your code here
adm_predict = pd.read_csv("data\Admission_Predict.csv")
adm_predict.shape

(385, 9)

Remove trailing spaces at the end of the column names if there are any.


In [65]:
# your code here
adm_predict.columns =adm_predict.columns.str.strip()
adm_predict.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [66]:
# your code here
adm_predict.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [67]:
# your code here
adm_predict.isnull().value_counts()

Serial No.  GRE Score  TOEFL Score  University Rating  SOP    LOR    CGPA   Research  Chance of Admit
False       False      False        False              False  False  False  False     False              385
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [68]:
# your code here
adm_predict = adm_predict.set_index(['Serial No.'], drop=False)
adm_predict.head()


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [69]:
# your code here
#adm_predict.shape
adm_predict['GRE&CGPA'] = adm_predict['GRE Score'].astype(str)+adm_predict['CGPA'].astype(str)
adm_predict['GRE&CGPA'].nunique()
#the new column GRE&CGPA also contains 385 unique records, which means it can be the identifier for individuals

385

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [70]:
adm_predict.set_index(['GRE&CGPA'], inplace=True, drop=True)


In [71]:

adm_predict.drop(columns = ['GRE Score', 'CGPA'], inplace=True)
adm_predict

,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE&CGPA,,,,,,,
3379.65,1,118,4,4.5,4.5,1,0.92
3168.0,2,104,3,3.0,3.5,1,0.72
3228.67,3,110,3,3.5,2.5,1,0.80
3148.21,4,103,2,2.0,3.0,0,0.65
3309.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...
3249.04,381,110,3,3.5,3.5,1,0.82
3259.11,382,107,3,3.0,3.5,1,0.84
3309.45,383,116,4,5.0,4.5,1,0.91


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [72]:
# your code here
adm_predict.reset_index(inplace =True)
adm_predict.head()

<bound method NDFrame.head of     GRE&CGPA  Serial No.  TOEFL Score  University Rating  SOP  LOR  Research  \
0    3379.65           1          118                  4  4.5  4.5         1   
1     3168.0           2          104                  3  3.0  3.5         1   
2    3228.67           3          110                  3  3.5  2.5         1   
3    3148.21           4          103                  2  2.0  3.0         0   
4    3309.34           5          115                  5  4.5  3.0         1   
..       ...         ...          ...                ...  ...  ...       ...   
380  3249.04         381          110                  3  3.5  3.5         1   
381  3259.11         382          107                  3  3.0  3.5         1   
382  3309.45         383          116                  4  5.0  4.5         1   
383  3128.78         384          103                  3  3.5  4.0         0   
384  3339.66         385          117                  4  5.0  4.0         1   

     Chan

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [74]:
# your code here
adm_predict = pd.read_csv("data\Admission_Predict.csv")
adm_predict.columns = adm_predict.columns.str.strip()


In [76]:
adm_predict.value_counts((adm_predict['CGPA']>9) & (adm_predict['Research']>0))

False    284
True     101
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [80]:
# your code here
CGPA_SOP = adm_predict[(adm_predict['CGPA']>9) & (adm_predict['SOP']<3.5)]
display(CGPA_SOP)
print("Mean chance of admit:", CGPA_SOP['Chance of Admit'].mean())

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
28,29,338,118,4,3.0,4.5,9.40,1,0.91
62,63,327,114,3,3.0,3.0,9.02,0,0.61
140,141,326,114,3,3.0,3.0,9.11,1,0.83
217,218,324,111,4,3.0,3.0,9.01,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84


Mean chance of admit: 0.8019999999999999


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [117]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """

    return (col - col.mean()) / col.std()


       

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [118]:
# your code here
# Apply the 'standardize' function to the columns
adm_predict['CGPA_std'] = standardize(adm_predict['CGPA'])
adm_predict['GRE_std'] = standardize(adm_predict['GRE Score'])
adm_predict['LOR_std'] = standardize(adm_predict['LOR'])
adm_predict

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,new_col,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,NaN,1.747900,1.753381,1.191647
1,2,316,104,3,3.0,3.5,8.00,1,0.72,NaN,-0.991211,-0.063367,0.076740
2,3,322,110,3,3.5,2.5,8.67,1,0.80,NaN,0.121034,0.455704,-1.038166
3,4,314,103,2,2.0,3.0,8.21,0,0.65,NaN,-0.642597,-0.236391,-0.480713
4,5,330,115,5,4.5,3.0,9.34,1,0.90,NaN,1.233279,1.147798,-0.480713
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,NaN,0.735259,0.628727,0.076740
381,382,325,107,3,3.0,3.5,9.11,1,0.84,NaN,0.851463,0.715239,0.076740
382,383,330,116,4,5.0,4.5,9.45,1,0.91,NaN,1.415886,1.147798,1.191647
383,384,312,103,3,3.5,4.0,8.78,0,0.67,NaN,0.303641,-0.409414,0.634193


We will generate the decision choice at random using the code below. Please run the cell.

In [119]:
# Libraries
from random import choices

In [121]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=adm_predict.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [129]:
# your code here
#creating the deciding column
adm_predict['deciding_column'] = adm_predict.lookup(adm_predict.index, decision_choice)


C:\Users\ruoxi\AppData\Local\Temp\ipykernel_9896\4204471051.py:3: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  adm_predict['deciding_column'] = adm_predict.lookup(adm_predict.index, decision_choice)


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,new_col,CGPA_std,GRE_std,LOR_std,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,NaN,1.747900,1.753381,1.191647,1.753381
1,2,316,104,3,3.0,3.5,8.00,1,0.72,NaN,-0.991211,-0.063367,0.076740,0.076740
2,3,322,110,3,3.5,2.5,8.67,1,0.80,NaN,0.121034,0.455704,-1.038166,0.121034
3,4,314,103,2,2.0,3.0,8.21,0,0.65,NaN,-0.642597,-0.236391,-0.480713,-0.642597
4,5,330,115,5,4.5,3.0,9.34,1,0.90,NaN,1.233279,1.147798,-0.480713,-0.480713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,NaN,0.735259,0.628727,0.076740,0.076740
381,382,325,107,3,3.0,3.5,9.11,1,0.84,NaN,0.851463,0.715239,0.076740,0.715239
382,383,330,116,4,5.0,4.5,9.45,1,0.91,NaN,1.415886,1.147798,1.191647,1.191647
383,384,312,103,3,3.5,4.0,8.78,0,0.67,NaN,0.303641,-0.409414,0.634193,0.634193


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [142]:
# your code here

def decision(row):
    if row['deciding_column'] > 0.8:
        return 1
    else:
        return 0

adm_predict['Decision'] = adm_predict.apply(decision, axis=1)
adm_predict.head(10)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,new_col,CGPA_std,GRE_std,LOR_std,deciding_column,Decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,NaN,1.747900,1.753381,1.191647,1.753381,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,NaN,-0.991211,-0.063367,0.076740,0.076740,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,NaN,0.121034,0.455704,-1.038166,0.121034,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,NaN,-0.642597,-0.236391,-0.480713,-0.642597,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,NaN,1.233279,1.147798,-0.480713,-0.480713,0
5,6,321,109,3,3.0,4.0,8.20,1,0.75,NaN,-0.659198,0.369192,0.634193,-0.659198,0
6,7,308,101,2,3.0,4.0,7.90,0,0.68,NaN,-1.157218,-0.755462,0.634193,0.634193,0
7,8,302,102,1,2.0,1.5,8.00,0,0.50,NaN,-0.991211,-1.274533,-2.153072,-1.274533,0
8,9,323,108,3,3.5,3.0,8.60,0,0.45,NaN,0.004829,0.542216,-0.480713,-0.480713,0
9,10,327,111,4,4.0,4.5,9.00,1,0.84,NaN,0.668856,0.888263,1.191647,0.668856,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [147]:
# your code here
adm_predict.value_counts(adm_predict['Decision']==1)

Decision
False    304
True      81
dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [151]:
# your code here
adm_predict.columns = adm_predict.columns.str.replace('.', '')


C:\Users\ruoxi\AppData\Local\Temp\ipykernel_9896\3699029703.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  adm_predict.columns = adm_predict.columns.str.replace('.', '')


In [153]:
adm_predict.columns = adm_predict.columns.str.replace(' ', '_')
adm_predict.columns = adm_predict.columns.str.lower()
adm_predict.head()

,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,new_col,cgpa_std,gre_std,lor_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,NaN,1.747900,1.753381,1.191647,1.753381,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,NaN,-0.991211,-0.063367,0.076740,0.076740,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,NaN,0.121034,0.455704,-1.038166,0.121034,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,NaN,-0.642597,-0.236391,-0.480713,-0.642597,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,NaN,1.233279,1.147798,-0.480713,-0.480713,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [163]:
# your code here
def adjusting(row):
    if row['university_rating']>=4:
        return row['gre_score'] +10
    else:
        return  row['gre_score']
    
adm_predict['adjusted_gre'] = adm_predict.apply(adjusting, axis = 1)

In [164]:
adm_predict.head()

,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,new_col,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,1,337,118,4,4.5,4.5,9.65,1,0.92,NaN,1.747900,1.753381,1.191647,1.753381,1,347
1,2,316,104,3,3.0,3.5,8.00,1,0.72,NaN,-0.991211,-0.063367,0.076740,0.076740,0,316
2,3,322,110,3,3.5,2.5,8.67,1,0.80,NaN,0.121034,0.455704,-1.038166,0.121034,0,322
3,4,314,103,2,2.0,3.0,8.21,0,0.65,NaN,-0.642597,-0.236391,-0.480713,-0.642597,0,314
4,5,330,115,5,4.5,3.0,9.34,1,0.90,NaN,1.233279,1.147798,-0.480713,-0.480713,0,340


In [167]:
bins = pd.cut(adm_predict['adjusted_gre'], 4, labels = ["Low", "Moderate", "High", "Very High"])
bins.value_counts()

Moderate     150
High          90
Very High     75
Low           70
Name: adjusted_gre, dtype: int64